# Task 4 AML : Sleeping Staging

In [1]:
import sys
!{sys.executable} -m pip install mne

The directory '/home/atroska/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/atroska/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 19.0.3, however version 20.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import psd_welch
from mne.time_frequency import tfr_stockwell
from mne import EpochsArray,create_info
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import balanced_accuracy_score

DEBUG:matplotlib:$HOME=/home/atroska
DEBUG:matplotlib:CONFIGDIR=/home/atroska/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'op

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:matplotlib:CACHEDIR=/home/atroska/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/atroska/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
train_eeg1 = pd.read_csv("train_eeg1.csv")
train_eeg2 = pd.read_csv("train_eeg2.csv")
train_emg = pd.read_csv("train_emg.csv")
train_labels = pd.read_csv("train_labels.csv")

test_eeg1 = pd.read_csv("test_eeg1.csv")
test_eeg2 = pd.read_csv("test_eeg2.csv")
test_emg = pd.read_csv("test_emg.csv")

sample = pd.read_csv("sample.csv")

In [4]:
train_eeg1_numpy=np.expand_dims(train_eeg1.to_numpy()[:,1:],-1)
train_eeg2_numpy=np.expand_dims(train_eeg2.to_numpy()[:,1:],-1)
train_emg_numpy=np.expand_dims(train_emg.to_numpy()[:,1:],-1)
train_labels_numpy=train_labels.to_numpy()[:,1:]

test_eeg1_numpy=np.expand_dims(test_eeg1.to_numpy()[:,1:],-1)
test_eeg2_numpy=np.expand_dims(test_eeg2.to_numpy()[:,1:],-1)
test_emg_numpy=np.expand_dims(test_emg.to_numpy()[:,1:],-1)

In [5]:
train_all_channels=np.concatenate((train_eeg1_numpy,train_eeg2_numpy,train_emg_numpy),axis=2)
train_all_channels.shape

(64800, 512, 3)

In [6]:
test_all_channels=np.concatenate((test_eeg1_numpy,test_eeg2_numpy,test_emg_numpy),axis=2)
test_all_channels.shape

(43200, 512, 3)

In [7]:
train_labels_numpy.shape

(64800, 1)

In [8]:
#train_labels_numpy[np.where(train_labels_numpy==1)]=0
#train_labels_numpy[np.where(train_labels_numpy==2)]=1
#train_labels_numpy[np.where(train_labels_numpy==3)]=2

In [9]:
def frequencies(epochs, pick="eeg"):
    WAVEFORMS = {"delta": [0.5, 4],
                 "theta": [4, 7],
                 "alpha":[8, 12],
                 "sigma": [12, 16],
                 "beta": [13, 30]
                }
    psds, freqs = psd_welch(epochs, picks=pick, fmin=0.5, fmax=30)
    print("PSDS : {}".format(psds))
    psds /= np.sum(psds, axis=-1, keepdims=True)
    results = []
    for fmin, fmax in WAVEFORMS.values():
        band = psds[:, :, (freqs < fmax) & (freqs >= fmin)].mean(axis=-1)
        results.append(band.reshape(len(psds), -1))
    results = np.nan_to_num(np.concatenate(results, axis=1))
    print("RESULTS: {}".format(results))
    return results
    

In [10]:
names=["eeg1", "eeg2", "emg"]
types=["eeg", "eeg", "emg"]
frequence = 128
info=create_info(ch_names=names, sfreq=frequence, ch_types=types)
print("Info: {}".format(info))
epochs=EpochsArray(np.reshape(train_all_channels,(64800,3, 512)), info=info)
print("Epochs: {}".format(epochs))



Info: <Info | 7 non-empty values
 bads: []
 ch_names: eeg1, eeg2, emg
 chs: 2 EEG, 1 EMG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: unspecified
 nchan: 3
 projs: []
 sfreq: 128.0 Hz
>
Not setting metadata
Not setting metadata
64800 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Epochs: <EpochsArray |  64800 events (all good), 0 - 3.99219 sec, baseline off, ~759.4 MB, data loaded,
 '1': 64800>


In [11]:
pipeline = make_pipeline(FunctionTransformer(frequencies, 
                                         validate=False),
                     RandomForestClassifier(n_estimators=200, 
                                            random_state=84,
                                            max_depth=30))

pipeline.fit(epochs, train_labels_numpy)

preds = pipeline.predict(epochs)
accuracy = balanced_accuracy_score(preds, train_labels_numpy)
print("Accuracy score: {}".format(accuracy))

Effective window size : 2.000 (s)
PSDS : [[[5.07699381e-09 6.22455711e-09 7.44003668e-09 ... 1.45009852e-11
   6.33334019e-11 1.72687905e-10]
  [1.01084925e-08 1.18638847e-08 9.91455434e-09 ... 3.55447462e-10
   1.98128431e-10 2.87889788e-10]]

 [[1.86769585e-08 2.77663109e-08 3.15614898e-08 ... 1.85055943e-10
   2.98866129e-10 5.70591081e-10]
  [2.79395649e-08 2.29713041e-08 2.09295514e-08 ... 1.80197625e-10
   4.08605586e-10 4.85999436e-11]]

 [[8.52093174e-08 3.42984595e-08 1.62575579e-08 ... 4.06051176e-11
   9.00213097e-11 1.42394314e-10]
  [1.58906850e-08 8.20907408e-09 5.87755969e-09 ... 3.00497926e-10
   2.26085677e-11 5.66586614e-11]]

 ...

 [[3.51804064e-08 5.55962532e-09 9.38590664e-09 ... 2.36039112e-10
   1.84050845e-10 4.99849801e-10]
  [2.49752564e-08 1.87712451e-08 3.61889693e-09 ... 3.16941007e-10
   2.97421588e-10 2.28986375e-10]]

 [[3.73393057e-09 1.13497770e-08 6.69791793e-09 ... 8.30799865e-10
   2.81849838e-10 5.16119561e-10]
  [1.12206557e-08 5.24376435e-09 1.0

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Effective window size : 2.000 (s)
PSDS : [[[5.07699381e-09 6.22455711e-09 7.44003668e-09 ... 1.45009852e-11
   6.33334019e-11 1.72687905e-10]
  [1.01084925e-08 1.18638847e-08 9.91455434e-09 ... 3.55447462e-10
   1.98128431e-10 2.87889788e-10]]

 [[1.86769585e-08 2.77663109e-08 3.15614898e-08 ... 1.85055943e-10
   2.98866129e-10 5.70591081e-10]
  [2.79395649e-08 2.29713041e-08 2.09295514e-08 ... 1.80197625e-10
   4.08605586e-10 4.85999436e-11]]

 [[8.52093174e-08 3.42984595e-08 1.62575579e-08 ... 4.06051176e-11
   9.00213097e-11 1.42394314e-10]
  [1.58906850e-08 8.20907408e-09 5.87755969e-09 ... 3.00497926e-10
   2.26085677e-11 5.66586614e-11]]

 ...

 [[3.51804064e-08 5.55962532e-09 9.38590664e-09 ... 2.36039112e-10
   1.84050845e-10 4.99849801e-10]
  [2.49752564e-08 1.87712451e-08 3.61889693e-09 ... 3.16941007e-10
   2.97421588e-10 2.28986375e-10]]

 [[3.73393057e-09 1.13497770e-08 6.69791793e-09 ... 8.30799865e-10
   2.81849838e-10 5.16119561e-10]
  [1.12206557e-08 5.24376435e-09 1.0

In [ ]:
print(preds)

In [ ]:
epochs_test=EpochsArray(np.reshape(test_all_channels,(43200,3,512)), info=info)
preds_test = pipeline.predict(epochs_test)
preds_test=preds_test #+1
test_id = sample.to_numpy()[:,0]
submission={"id": test_id,
           "y": preds_test.flatten()}
submission=pd.DataFrame(submission)
print(submission.head())
submission.to_csv("y_test_1.csv")